In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 38049732
paper_name = 'li_wang_2023' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [7]:
original_data = pd.read_excel('raw_data/TableS1.xlsx', sheet_name='Sheet1')

In [8]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 294 x 10


In [9]:
original_data.head()

,NO,Array ORF,Array Name,Normalized colony size (EXPERIMENT),Normalized colony std. dev. (EXPERIMENT),Normalized colony size (CONTROL),Normalized colony std. dev. (CONTROL),Score,Score stdev,p-Value
0,1,YMR283C,RIT1,0.38,0.01,1.04,0.03,-0.66,0.01,0
1,2,YDR049W,VMS1,0.38,0.00,0.98,0.02,-0.60,0.00,0
2,3,YJR075W,HOC1,0.69,0.01,1.21,0.02,-0.52,0.01,0
3,4,YBL047C,EDE1,0.40,0.02,0.92,0.01,-0.52,0.02,0
4,5,YDR068W,DOS2,0.42,0.04,0.94,0.02,-0.52,0.04,0


In [10]:
original_data['orf'] = original_data['Array ORF'].astype(str)

In [11]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [12]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [13]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [NO, Array ORF, Array Name, Normalized colony size (EXPERIMENT), Normalized colony std. dev. (EXPERIMENT), Normalized colony size (CONTROL), Normalized colony std. dev. (CONTROL), Score, Score stdev, p-Value, orf]
Index: []


In [14]:
original_data['data'] = original_data['Score']

In [15]:
original_data.set_index('orf', inplace=True)

In [16]:
original_data = original_data[['data']].copy()

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(291, 1)

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [22264]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,22264
data_type,value
orf,
YAL013W,0.57
YAL022C,0.23
YAR002W,0.39
YBL007C,-0.23
YBL024W,-0.20


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22264
,data_type,value
gene_id,orf,
11,YAL013W,0.57
20,YAL022C,0.23
62,YAR002W,0.39
95,YBL007C,-0.23
112,YBL024W,-0.20


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22264          
data_type       value    valuez
gene_id orf                    
11      YAL013W  0.57  6.912566
20      YAL022C  0.23  2.755393
62      YAR002W  0.39  4.711709
95      YBL007C -0.23 -2.869017
112     YBL024W -0.20 -2.502208

# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')